#### 这是草稿版本，用来学习和测试的，请看另外版本。

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
def loadDataSet(filename):
    ### get dataset
    dataMat=[]
    labelMat=[]
    with open('testSet.txt') as f:
        for line in f.readlines():
            linestrip = line.strip().split('\t')
            dataMat.append([float(linestrip[0]), float(linestrip[1])])
            labelMat.append(float(linestrip[2]))
    return dataMat, labelMat

In [75]:
data,label = loadDataSet('testSet.txt')

In [78]:
data = np.array(data)

In [79]:
label=np.array(label)

In [37]:
b=0

In [40]:
m,n = np.shape(dataMatrix)

In [51]:
alphas = np.mat(np.zeros((10,1)))

In [5]:
from sklearn.svm import SVC

In [7]:
?SVC

In [12]:
data.shape

(100, 2)

In [20]:
np.zeros(10).shape

(10,)

In [21]:
a = np.array([[1,2],[1,2]])

In [22]:
a * a 

array([[1, 4],
       [1, 4]])

In [23]:
a @ a

array([[3, 6],
       [3, 6]])

In [37]:
b = np.array([1,2])

In [38]:
b[0]

1

In [27]:
b.shape

(2,)

In [32]:
b.T @ b

5

In [170]:
class SVM():
    def __init__(self, kernel='linear', epsilon = 0.001, C = 1.0):
        """
        C:惩罚系数
        """
        self.kernel = kernel
        self.C = C
        self.epsilon = epsilon
    
    def _init_parameters(self, features, labels):
        """
        (features,labels)  np.array
        
        """
        self.X = features
        self.Y = labels
        self.b = 0.0
        self.n = features.shape[0]
        self.p = features.shape[1]
        self.alpha = np.zeros(self.n)
        self.E = [self._get_E(i) for i in range(self.n)]
        self.Max_iteration = 20000
    
    def _gx(self,i):
        """
        i: 第i+1个样本(start from 0 and end with N-1)
        """
        g_x_i = (self.alpha * self.Y).T @ (self.X @ self.X[i,:]) + self.b
        return g_x_i
    
    def _get_E(self,i):
        """
        i: 第i+1个样本(start from 0 and end with N-1)
        """
        E_i = self._gx(i) - self.Y[i]
        return E_i
    
    def _satisfy_KKT(self, i):
        ygx = self.Y[i] * self._gx(i)
        if abs(self.alpha[i]) <= self.epsilon:
            return ygx >= 1
        elif abs(self.alpha[i] - self.C) <= self.epsilon:
            return ygx <= 1
        else:
            return abs(ygx-1) <= self.epsilon
        
    def is_stop(self):
        for i in range(self.n):
            satisfy = self._satisfy_KKT(i)
            
            if not satisfy:
                return False
        return True
    
    def _select_two_alpha(self):
        index_i = list(range(self.n))
        i_list_1 = list(filter(lambda i: self.alpha[i] > 0 + self.epsilon and self.alpha[i] < self.C - self.epsilon, index_i))
        i_list_2 = list(set(index_i) - set(i_list_1))
                
        for i in i_list_1:
            ### 第一步
            if self._satisfy_KKT(i):
                continue
            E_1 = self._get_E(i)
            E2_E1 = np.abs(self.E - E_1)
            j = E2_E1.argmax()
            #print(i,j)
            return i, j
        for i in i_list_2:
            if self._satisfy_KKT(i):
                continue
            E_1 = self._get_E(i)
            E2_E1 = np.abs(self.E - E_1)
            j = E2_E1.argmax()
            #print(i,j)
            return i, j
        
#             j_max = 999
#             diff_max
#             ### 第二步
#             for j in index_i:
#                 if i == j:
#                     continue
#                 E_2 = self._get_E(j)
#                 diff = abs(E_2 - E_1)
#                 if diff > diff_max:
#                     diff_max = diff
#                     j_max = j
#             return i, j_max
    
#     def _K_(self):
#         """
#         暂时先不考虑核函数
#         """
    def train(self, features, labels):
        
        self._init_parameters(features, labels)
        for times in range(self.Max_iteration):
            #print(times)
            if self.is_stop():
                break
            i1, i2 = self._select_two_alpha()
            
            if self.Y[i1] == self.Y[i2]:
                L = max(0, self.alpha[i2] + self.alpha[i1] - self.C)
                H = min(self.C, self.alpha[i2] + self.alpha[i1])
            else:
                L = max(0, self.alpha[i2] - self.alpha[i1])
                H = min(self.C, self.C + self.alpha[i2] - self.alpha[i1])
            E1 = self.E[i1]
            E2 = self.E[i2]
            ###要改kernel这里得改
            eta = self.X[i1,:] @ self.X[i1,:] + self.X[i2,:] @ self.X[i2,:] -2 * self.X[i1,:] @ self.X[i2,:]
            alpha2_new_unre = self.alpha[i2] + self.Y[i2] * (E1 - E2) / eta
            
            if alpha2_new_unre > H:
                alpha2_new = H
            elif alpha2_new_unre < L:
                alpha2_new = L
            else:
                alpha2_new = alpha2_new_unre
            alpha1_new = self.alpha[i1] + self.Y[i1] * self.Y[i2] * (self.alpha[i2] - alpha2_new)
            
            
            ###要改kernel这里得改
            b1_new = -E1 - self.Y[i1]* (self.X[i1,:] @ self.X[i1,:]) * (alpha1_new - self.alpha[i1]) - \
                    self.Y[i2] * (self.X[i2,:] @ self.X[i1,:]) * (alpha2_new - self.alpha[i2]) + self.b 
            ###要改kernel这里得改
            b2_new = -E2 - self.Y[i1]* (self.X[i1,:] @ self.X[i2,:]) * (alpha1_new - self.alpha[i1]) - \
                    self.Y[i2] * (self.X[i2,:] @ self.X[i2,:]) * (alpha2_new - self.alpha[i2]) + self.b 
            
            if alpha1_new > 0 + self.epsilon and alpha1_new < self.C - self.epsilon:
                b_new = b1_new
            elif alpha2_new > 0 + self.epsilon and alpha2_new < self.C - self.epsilon:
                b_new = b2_new
            else:
                b_new = (b1_new + b2_new)/2
            
            self.alpha[i1] = alpha1_new
            self.alpha[i2] = alpha2_new
            self.b = b_new
            ### 和原博主不一样
            self.E = [self._get_E(i) for i in range(self.n)]
        #print(times)
    
    def _predict_(self, feature):
        """
        这里feature是p X 1 维的 （单个样本）
        """
        result = (self.alpha * self.Y).T @ (self.X @ feature) + self.b
        if result >=0 :
            return 1
        return -1
    def predict(self, features):
        results = []
        
        for i in range(features.shape[0]):
            results.append(self._predict_(features[i,:]))
        return np.array(results)

In [178]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [179]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.3, random_state = 1103)

In [180]:
SVM_my = SVM()
SVC_sk = SVC()

In [181]:
import warnings
warnings.filterwarnings("ignore")

In [182]:
%timeit SVM_my.train(X_train, y_train)
result_my = SVM_my.predict(X_test)

19999
19999
19999
19999
19999
19999
19999
19999
7.2 s ± 194 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [183]:
%timeit SVC_sk.fit(X_train, y_train)
result_sk = SVC_sk.predict(X_test)

416 µs ± 9.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [184]:
result_sk == result_my

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])